<a href="https://colab.research.google.com/github/LucianPopaLVP/Deep-Learning-Project--Dog-Vision/blob/main/dog_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🐶 End-to-end Multi-class Dog Breed Classification

This notebook buils an end-to-end multi-class image classifier using Tensorflow 2.0 and TensorFlow Hub.

## 1. Problem

Identifying the breed of a dog given an image of a dog.

## 2. Data

The data used is from Kaggle's dog bread identification competition: https://www.kaggle.com/c/dog-breed-identification/overview

## 3. Evaluation

Multi-Class Log Loss: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html

## 4. Features

Info about data:
* Dealing with images (unstructured data) so is probably best to use deep learning/transfer learning;
* There are 120 breeds of dogs (this means there are 120 different classes);
* Theer are ~ 10,000 + images in the training set(these images have labels) and ~ 10,000 images in the test set(no labels);